In [0]:
%%sh
export PYTHONPATH=$SPARK_HOME/python/:$SPARK_HOME/python/lib/py4j-0.10.7-src.zip:$PYTHONPATH
python  ~/bdp/application/aaintdatapipeline/core/pipeline/doctor.py readavro -k raw/app-int.advancedreview.topic.v1/insert/_default_partition=2019-12/aidpc896494dfa40e6cc993824da9404b451.avro


In [0]:
%%sh
aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_INHOUSE_TOPIC_PREDICTIONS/
# ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2020-04-24/   --recursive --human --summarize | tail -5

  #  --recursive --human --summarize | tail -5

# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-04-29/   --recursive --human --summarize | tail -5


 

# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/  --recursive --human --summarize | tail -5


# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-05-12/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-05-12/


# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-07-11/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-07-11/


In [0]:




# spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=2.0.3/platform=2/process_date=2020-08-19/").where("country in ('IN') and date_format(review_datetime, 'yyyy-MM-dd') ='2020-08-19' ").show(2)


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-raw-advancedreview/raw/advancedreview.topic.v1/insert/_default_partition=2020-10/ | sort -n


In [0]:


df = spark.read.option("basePath", "s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2").load("s3://b2c-prod-advanced-review/oss/ADVRVW_RELEVANT_TERMS/version=1.0.1/platform=2/process_date=2020-09-2{4,5,6}").cache()
df.createOrReplaceTempView("temp")
spark.sql("select * from temp where app_id = 284882215 and country = 'JP'").show()

In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
from datetime import datetime,timedelta
import datetime

# test_date = ['2019-07-0'+ str(a) for a in xrange(1,10)] + ['2019-07-1'+str(a) for a in xrange(0,6)]
numdays=122
base = datetime.date.today()- datetime.timedelta(days=23)
date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]

                            
#and review_datetime='2019-07-14T00:00:00.000000Z'
def check_data_in_process_date_group_by(pd):
    print pd
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]

    raw_process_date_path = "s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2" \
                        "/process_date={}/".format(pd)
    unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview" \
                            ".topic.v1/"
    for language in language_list:
        for country in country_list:

            t = unittest.TestCase('run')
            raw_result = spark.read.parquet(raw_process_date_path).filter("country='{}' and language='{}'".format(country,language))
            raw_result = raw_result.withColumn("record_date", raw_result['review_datetime'].cast(DateType()))
            raw_result = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
            raw_data_row = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id")).collect()

            unified_result = spark.read.parquet(unified_process_date_path)
            unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
            unified_data_row = unified_result.filter("process_date='{}' and country_code='{}' and language='{}'".format(pd,country,language)).collect()

            range_len = len(raw_data_row)
            range_len_unified=len(unified_data_row)
            review_id_list = []
            for x in range(0,range_len):
                review_id_list.append(raw_data_row[x][1])
            # print len(review_id_list)

            review_id_list_unified = []
            # print range_len_unified
            for x in range(0,range_len_unified):
                review_id_list_unified.append(unified_data_row[x][9])
            # print len(review_id_list_unified)
            l= [x for x  in review_id_list if x not in review_id_list_unified]
            # print '!!!', len(l) , "different is : " , l 
            for x in l:
                row = unified_result.filter("review_id={}".format(x)).select("process_date").collect()[0]
                _date = datetime.datetime.strptime(pd, "%Y-%m-%d").date()
                if(row[0]<_date):
                    print 'compare current process date' , pd , ' ~~~ unified process date',  row
        print '~~~~~~~~~~~'

    
    
    
    # print list(set(review_id_list) & set(review_id_list_unified)) 

    print 'no error'
                                                                                   
for x in date_list:
    check_data_in_process_date_group_by(x)

In [0]:

from pyspark.sql.functions import array_contains
from pyspark.sql.functions import col


unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/date=2019-10-17/part-20191017.c000.gz.parquet"
df=spark.read.parquet(unified_process_date_path)
df.printSchema()

df=df.withColumn("contains_1",df.select(array_contains(col("topic_ids"),1)))
df.show()


In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -p 5432 -U citus_bdp_usage_qa -d aa_citus_db << EOF
SET search_path=advancedreview;
--\d advancedreview_topic_fact_v3;
--\d advancedreview_term_fact_v3;
BEGIN;
-- SELECT count(*) FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='US' and language='en' and array_length(topic_ids, 1)>=2 and topic_ids@>'{1}';
--SELECT * FROM  advancedreview.advancedreview_topic_fact_v2 where country_code='IN' and review_id='4395695885';
-- select * from advancedreview_topic_fact_v2 where  cardinality(topic_ids)=0 order by date desc limit 3;
-- select * from advancedreview_topic_fact_v2 where topic_ids <@ ARRAY[10]::smallint[] limit 10 ;
-- select _identifier, count(*) from advancedreview_topic_fact_v2 group by _identifier order by _identifier ;
-- select date, count(*) from advancedreview_term_fact_v3 where date in ('2020-05-09', '2020-05-10', '2020-05-11','2020-05-12') group by date;
-- select date, count(*) from advancedreview_topic_fact_v3 where date in ('2020-05-10','2020-05-11','2020-05-12') group by date;
select date, count(1), country_code from advancedreview_topic_fact_v3 where date in ('2020-12-14','2020-12-13', '2020-12-12', '2020-12-11') group by country_code, date order by country_code, date ;
--select date, count(1), country_code from advancedreview_term_fact_v3 where date in ('2020-12-13', '2020-12-12', '2020-12-11')  group by country_code, date order by country_code, date ;

COMMIT;
EOF 



In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
import datetime

raw_process_date_path = "s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2" \
                        "/process_date={}/count/lang"
unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview" \
                            ".topic.v1/fact/process_date={}/"
                            

def check_data_in_process_date_group_by():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]
    # test_date = datetime.date.today()- datetime.timedelta(days=2)
    numdays=116
    start_date='2019-07-01'
    end_date='2019-10-24'
    base = datetime.date.today()- datetime.timedelta(days=32)
    expected_date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
    for test_date in expected_date_list:
        for country in country_list:
            for language in language_list:
                print 'started to test: ', country, language , test_date
                raw_path=raw_process_date_path.format(test_date)
                raw_result = spark.read.parquet(raw_path).filter("country='{}' and language='{}'".format(country,language))
                raw_result = raw_result.withColumn("record_date", raw_result['review_datetime'].cast(DateType()))
                raw_result = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
                raw_data_row = sorted(raw_result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).collect())

            # print raw_data_row
                unified_path=unified_process_date_path.format(test_date)
                unified_result = spark.read.parquet(unified_path)
                unified_result = unified_result.withColumn("record_date_unified", unified_result['date'].cast(DateType()))
                unified_data_row = sorted(unified_result.filter("country_code='{}' and language='{}'".format(country,language)).groupBy("record_date_unified").count().collect())

            # print unified_data_row
            
            
                range_len = len(raw_data_row)
                print range_len
                for x in range(0, range_len):
                # print raw_data_row[x][0], type(raw_data_row[x][0])
                # print unified_data_row[x][0], type(unified_data_row[x][0])
                    t.assertEqual(raw_data_row[x][0], unified_data_row[x][0],
                     'date is not equals {} ~ {}, debug info {} \n {} '.format(raw_data_row[x][0],
                                                                              unified_data_row[x][0],
                                                                              raw_path,
                                                                              unified_path))
                # print raw_data_row[x][1], type(raw_data_row[x][1])
                # print unified_data_row[x][1], type(unified_data_row[x][1])
                    t.assertEqual(raw_data_row[x][1], unified_data_row[x][1],
                     'number is not equals {} ~ {}, debug info {} \n {} '.format(raw_data_row[x][1],
                                                                              unified_data_row[x][1],
                                                                              raw_path,
                                                                              unified_path))
    print 'no error'
                           
                           

check_data_in_process_date_group_by()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/



In [0]:

# raw_path="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2019-10-25/"
path="s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-02-03"
spark.read.parquet(path).filter("review_id='5481813221'").show(20,False)

path="s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/process_date=2020-02-03"
spark.read.parquet(path).filter("review_id='5481813221'").show(20,False)


path1="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2020-02-03"
spark.read.parquet(path1).filter("review_id='5481813221'").show(20,False)


# path2="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2020-02-03"
# spark.read.parquet(path2).filter("review_id='5121372801'").show()

#  review_id  | product_id | product_version | country_code | language | rating |    date    | topic_ids | device_code |    _identifier     | _disable_idx_4_query 
# ------------+------------+-----------------+--------------+----------+--------+------------+-----------+-------------+--------------------+----------------------
#  5121372801 | 1461479554 | 1.4.6           | US           | en       |      3 | 2019-11-11 | {2}       | ios-all     | 220191128041020689 |                    0

# print c1 
# print c2

In [0]:

df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.term.v1/fact/process_date=2020-02-03").filter("review_id=5458214386").show()
df1=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.term.v1/fact/process_date=2020-02-03").filter("review_id=5458214386").show()


In [0]:

import datetime
numdays=116
start_date='2019-07-01'
end_date='2019-12-20'
base = datetime.date.today()- datetime.timedelta(days=150)
expected_date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
print expected_date_list

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v2/fact/


In [0]:

import unittest
from pyspark.sql.functions import count

def count_review_id_test():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]
    for language in language_list:
        for country in country_list:
            print country
            result = spark.read.parquet("s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2020-02-03/")
            raw_count = result.filter("country='{}' and language='{}'".format(country,language)).select("review_id").distinct().count()
            result1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/process_date=2020-02-03")
            unified_count = result1.filter("country_code='{}' and language='{}'".format(country,language)).select("review_id").distinct().count()
            print 'raw count is : ', raw_count
            print 'unified_count is : ', unified_count
            t.assertEqual(raw_count,unified_count, "raw data is not equals with unifed data {} - {} ".format(raw_count,unified_count))
    
count_review_id_test()

In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg


unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/"
def check_rating_is_1_to_5():
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]

    for country in country_list:
        print 'test country : ' , country
        t = unittest.TestCase('run')
        unified_result = spark.read.parquet(unified_process_date_path)
        unified_data_row = unified_result.filter(
            "process_date='2020-02-03' and country_code='{}' and language='en'".format(country)).collect()

        range_len = len(unified_data_row)
        for x in range(0, range_len):
            result= unified_data_row[x].asDict()["rating"]
            t.assertTrue(result in [1,2,3,4,5], "rating value is not correct {}".format(result))
    print 'pass'
check_rating_is_1_to_5()


In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg

unified_process_date_path = "s3://b2c-prod-data-pipeline-unified-advancedreview/unified/advancedreview.topic.v1/fact/"
def check_is_topic_is_1_to_11():

    unittest.TextTestRunner(verbosity=2)
    t = unittest.TestCase('run')
    unified_result = spark.read.parquet(unified_process_date_path)
    unified_result = unified_result.withColumn("record_date", unified_result['date'].cast(DateType()))
    unified_data_row = unified_result.filter(
        "process_date='2020-02-03' and country_code='US' and language='en'").collect()

    range_len = len(unified_data_row)
    for x in range(0, range_len):
        array_result= unified_data_row[x].asDict()["topic_ids"]
        for i in array_result:
            t.assertTrue(i in range(1,12), "topic_ids value is not correct {} ".format(i))
    print 'pass'
check_is_topic_is_1_to_11()


In [0]:

import unittest
from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
from datetime import datetime,timedelta
import datetime
from aaintdatapipeline.core.conf import Conf
from aaintdatapipeline.core.fs.device import S3Bucket
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import get_mkt_unified_path, check_parquet_exist

conf = Conf(
    bucket_name="b2c-prod-data-pipeline-unified-advancedreview"
)

def test_date_completeness():
    device_code='ios-all'
    t = unittest.TestCase('run')
    numdays=136
    base = datetime.date.today()- datetime.timedelta(days=2)
    date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
    path_fail, data_fail = dict(), dict()
    path_fail[device_code], data_fail[device_code] = list(), list()
    for date in date_list:
        unified_parquet_path = "unified/app-int.advancedreview.topic.v1/fact/process_date={}/".format(date)
        files = S3Bucket(conf=conf).all(unified_parquet_path)
        if files:
            if check_parquet_exist(files):
                data_fail[device_code].append(date)
        else:
            path_fail[device_code].append(date)
    if not path_fail[device_code]:
        path_fail.pop(device_code)
    if not data_fail[device_code]:
        data_fail.pop(device_code)
    t.assertFalse(data_fail or path_fail, msg='{}~{}'.format(data_fail, path_fail))
    print 'passed'
test_date_completeness()


In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count
from pyspark.sql import Row

def test_count():
    t = unittest.TestCase('run')
    country_list=["US"]#, "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en"]#,"ja","ko"]
    sql="SELECT distinct (review_id) FROM  advancedreview.advancedreview_topic_fact_v2  where device_code='ios-all';"
    raw_path_1="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.1_dedupe/platform=2/"
    raw_path_2="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/"

    for language in language_list:
        for country in country_list:
            result_1 = spark.read.parquet(raw_path_1).select("review_id").distinct()
            result_2 = spark.read.parquet(raw_path_2).filter("process_date>='2019-11-12' and process_date<='2019-11-26'").select("review_id").distinct()
            result = result_1.union(result_2).select("review_id").distinct()
            db_result = get_citus_connection(urn, sql)
            rows_result=[]
            for row in db_result:
                rows_result.extend(Row(review_id_from_db=row))
            new_df=spark.createDataFrame(rows_result)
            # print 'raw count is : ', result
            # print 'db is : ', db_result[0][0]
            result.select("review_id").subtract(new_df).show()


# s1=x1.select("review_id")
# s2=x2.select("review_id")
# s1.union(s2).subtract(s1.intersect(s2)).show()

test_count()

In [0]:
%%sh
aws s3 ls s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/process_date=2020-02-12/

In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count
from pyspark.sql import Row

def test_count():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]
    # sql="SELECT count(distinct review_id) FROM  advancedreview.advancedreview_topic_fact_v3  where device_code='ios-all';"
    sql="SELECT count(distinct review_id) FROM  advancedreview.advancedreview_topic_fact_v3  where country_code='{}' and language='{}' and device_code='ios-all';"

    raw_path="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/"
    for language in language_list:
        for country in country_list:
            result = spark.read.parquet(raw_path).filter("country='{}' and language='{}'".format(country,language)).select("review_id").distinct().count()
            # result = spark.read.parquet(raw_path).select("review_id").distinct().count()
            print language, country
            # db_result = get_citus_connection(urn, sql)
            db_result = get_citus_connection(urn, sql.format(country,language))
            print 'raw count is : ', result
            
            # path2="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/"
            # print 'another part row...: ' ,spark.read.parquet(path2).filter("process_date>='2019-11-12' and process_date<='2019-11-25'").select("review_id").distinct().count()

            print 'db is : ', db_result[0][0]
            t.assertEqual(result,db_result[0][0], "raw data is not equals with unifed data {} - {} ".format(result,db_result[0][0]))

    print 'pass'

test_count()

In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count
import datetime

def datetime_to_string(date_time, convert_format="%Y-%m-%d"):
    return datetime.datetime.strftime(date_time, convert_format)

def check_date_completeness():
    numdays=148
    start_date='2019-07-01'
    end_date='2019-11-27'
    base = datetime.date.today()- datetime.timedelta(days=2)
    expected_date_list = [str(base - datetime.timedelta(days=x)) for x in range(numdays)]
    # print expected_date_list
    t = unittest.TestCase('run')

    date_fail = dict()
    actual_date_list = []
    country_code_list = ['US', 'GB', 'CA', 'AU', 'IN', 'JP', 'KR']
    language_list=["en","ja","ko"]
    device_code='ios-all'
    for country_code in country_code_list:
        print country_code
        for language in language_list:
            date_fail[device_code] = list()
            sql = "SELECT distinct date FROM  advancedreview.{} where country_code='{}' and language='{}' and device_code='ios-all' and date between '{}' and '{}'".format("advancedreview_topic_fact_v3", country_code, language, start_date, end_date)
            date_list = get_citus_connection(urn, sql)
            for item in date_list:
                actual_date_list.append(datetime_to_string(item[0]))
            for date in expected_date_list:
                if str(date) not in actual_date_list:
                    date_fail[device_code].append(date)
            if not date_fail[device_code]:
                date_fail.pop(device_code)
            t.assertFalse(date_fail, msg='{}'.format(date_fail))
    print 'pass'

check_date_completeness()


In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count

def check_topic_mutually_exclusive_count():
    t = unittest.TestCase('run')
    country_list=["US", "GB", "CA", "AU", "IN", "JP", "KR"]
    language_list=["en","ja","ko"]
    sql="SELECT count(*) FROM  advancedreview.advancedreview_topic_fact_v3 where country_code='{}' and language='{}' and array_length(topic_ids, 1)>=2 and topic_ids@>'{{1}}';"
    for language in language_list:
        for country in country_list:
            print country, language
            db_result = get_citus_connection(urn, sql.format(country,language))
            print 'db is : ', db_result[0][0]
            t.assertEqual(0,db_result[0][0], "find topic contains 1 and others - {} ".format(db_result[0][0]))
    print 'pass'

check_topic_mutually_exclusive_count()

In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count

def check_topic_id_list_is_empty():
    t = unittest.TestCase('run')
    sql="select count(1) from advancedreview.advancedreview_topic_fact_v3 where cardinality(topic_ids)=0;"
    db_result = get_citus_connection(urn, sql)
    t.assertEqual(0,db_result[0][0], "find empty topic list - {} ".format(db_result[0][0]))
    print 'pass'

check_topic_id_list_is_empty()

In [0]:

from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_aso_citus_connection)
from aaintdatapipeline.application.app_qa.data_validation_v1.constants import urn
from aaintdatapipeline.application.app_qa.data_validation_v1.utils import (
    get_citus_connection)
import unittest
from pyspark.sql.functions import count

def check_topic_id_list_is_empty():
    t = unittest.TestCase('run')
    sql="select * from advancedreview.advancedreview_topic_fact_v3 where product_id='1318292856' and country_code='US' order by date desc limit 3"
    db_result = get_citus_connection(urn, sql)
    print db_result

check_topic_id_list_is_empty()

In [0]:
%%sh

aws s3 ls s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/
# spark.read.parquet(path2).filter("review_id='5121372801'").show()



In [0]:

from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
import datetime
path="s3://b2c-prod-advanced-review/oss/ADVRVW_TOPIC_EFFECTIVE_PREDICTIONS/version=1.0.0/platform=2/"
raw_result=spark.read.parquet(path).filter("app_id='1318292856' and country='US' ")
raw_result = raw_result.withColumn("record_date", raw_result['review_datetime'].cast(DateType()))
raw_result = raw_result.groupBy("record_date", "review_id").agg(count("review_id").alias("count_review_id"))
raw_data_row = sorted(raw_result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).collect())
print raw_data_row



In [0]:

from pyspark.sql.types import DateType
from pyspark.sql.functions import count, avg
import datetime

path="s3://b2c-prod-data-pipeline-unified-advancedreview/unified/app-int.advancedreview.topic.v1/fact/"
unified_data=spark.read.parquet(path).filter("app_id='1318292856'")
unified_result = unified_data.groupBy("review_date", "review_id").agg(count("review_id").alias("count_review_id"))
raw_data_row = sorted(raw_result.groupBy("record_date").agg(count("count_review_id").alias("total_review")).collect())
print raw_data_row